<a href="https://colab.research.google.com/github/salmonin-o3o/ESAA_study/blob/main/250314_markdown.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

핸즈온
# CH 07. 앙상블 학습과 랜덤 포레스트
'대중의 지혜'처럼 일련의 예측기로부터 예측을 수집하면 가장 좋은 모델 하나보다 더 좋은 예측을 얻을 수 있다. 일련의 예측기를 앙상블이라고 하고, 이를 앙상블 학습이라고 한다. 그리고 앙상블 학습 알고리즘을 앙상블 방법이라고 한다.

예를 들면, 랜덤 포레스트가 있으며, 배깅, 부스팅, 스태킹 등도 있다.

## 7.1 투표 기반 분류기
분류기 여러 개를 훈련시켜서, 각 분류기의 예측을 모아 가장 많이 선택된 클래스를 예측한다. 이렇게 다수결 투표로 정해지는 분류기를 직접 투표 분류기라고 한다.

각 분류기가 약한 학습기일지라도 충분하게 많고 다양하다면 앙상블은 강한 학습기가 될 수 있다.(큰 수의 법칙에 의해)

+) 앙상블 방법은 예측기가 가능한 한 서로 독립적일 때 최고의 성능을 발휘한다.

다음은 여러 분류기를 조합하여 사이킷런의 투표 기반 분류기(VotingClassifier)를 만들고 훈련시키는 코드이다.

In [1]:
import warnings
warnings.filterwarnings('ignore')

# import package
import numpy as np
import os

# 5장에서 소개한 moons dataset 불러오기
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

X, y = make_moons(n_samples=100, noise=0.15)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard')
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [3]:
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.85
RandomForestClassifier 0.95
SVC 0.95
VotingClassifier 0.95


모든 분류기가 클래스의 확률을 예측할 수 있으면(predict_prova()), 개별 분류기의 예측을 평균 내어 확률이 가장 높은 클래스를 예측할 수 있다. 이를 간접 투표라고 한다. -> voting="soft"

## 7.2 배깅과 페이스팅
또 다른 방법은 같은 알고리즘을 사용하고 훈련 세트의 서브셋을 무작위로 구성하여 분류기를 각기 다르게 학습시키는 것이다. 이때 훈련 세트에서 중복을 허용하여 샘플링하는 방식을 배깅이라 하며, 중복을 허용하지 않고 샘플링 하는 방식을 페이스팅이라고 한다.

모든 예측기가 훈련을 마치면 앙상블은 모든 예측기의 예측을 모아서 새로운 샘플에 대한 예측을 만든다. 수집 함수는 전형적으로 분류일 때는 통계적 최빈값이고, 회귀에 대해서는 평균을 계산한다.

### 7.2.1 사이킷런의 배깅과 페이스팅
사이킷런은 배깅과 페이스팅을 위해 간편한 API로 구성된 BaggingClassifier를 제공한다.

In [4]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=50, bootstrap=True, n_jobs=-1)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

앙상블은 비슷한 편향에서 더 작은 분산을 만든다.

전반적으로 배깅이 더 나은 모델을 만들지만, 시간과 CPU 파워에 여유가 있다면 교차 검증으로 배깅과 페이스팅을 모두 평가해서 더 나은 쪽을 선택하는 것이 좋다.

### 7.2.2 oob 평가
배깅을 사용하면 어떤 샘플은 한 예측기를 위해 여러 번 샘플링되고 어떤 것은 전혀 선택되지 않을 수 있다. 이때 선택되지 않은 훈련 샘플의 나머지를 oob 샘플이라고 부른다.

예측기가 훈련되는 동안에는 oob 샘플을 사용하지 않으므로 이를 이용해서 평가할 수 있다. 앙상블의 평가는 각 예측기의 oob 평가를 평균하여 얻게 된다.

oob_score=True로 지정하여 수행할 수 있으며, 평가 점수 결과는 oob_score_ 변수에 저장되어 있다.

In [5]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    bootstrap=True, n_jobs=-1, oob_score=True)

bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.925

In [6]:
from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.95

oob 샘플에 대한 결정 함수의 값도 obb_decision_fuction 변수에서 확인할 수 있다. 이 경우 결정 함수는 각 훈련 샘플의 클래스 확률을 반환한다.

In [8]:
bag_clf.oob_decision_function_

array([[0.        , 1.        ],
       [0.        , 1.        ],
       [0.0104712 , 0.9895288 ],
       [0.32571429, 0.67428571],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.91954023, 0.08045977],
       [0.10382514, 0.89617486],
       [0.71111111, 0.28888889],
       [0.        , 1.        ],
       [0.91463415, 0.08536585],
       [0.70555556, 0.29444444],
       [0.        , 1.        ],
       [0.96703297, 0.03296703],
       [1.        , 0.        ],
       [0.95081967, 0.04918033],
       [0.        , 1.        ],
       [0.99438202, 0.00561798],
       [0.49479167, 0.50520833],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.984375  , 0.015625  ],
       [0.        , 1.        ],
       [0.99479167, 0.00520833],
       [0.33898305, 0.66101695],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.47959184, 0.52040816],
       [1.        , 0.        ],
       [0.93229167, 0.06770833],
       [0.

## 7.3  랜덤 패치와 랜덤 서브스페이스
BaggingClassifier는 특성 샘플링도 지원한다. 샘플링은 max_features, bootstrap_features 두 매개변수로 조절된다.

이 기법은 특히 (이미지와 같은) 매우 고차원의 데이터셋을 다룰 떄 유용하다. 훈련 특성과 샘플을 모두 샘플링하는 것을 랜덤 패치 방식이라고 한다. 훈련 샘플을 모두 사용하고 특성은 샘플링하는 것을 랜덤 서브스페이스 방식이라고 한다.

## 7.4 랜덤 포레스트
랜덤 포레스트는 일반적으로 배깅 방법(또는 페이스팅)을 적용한 결정 트리의 앙상블이다. 전형적으로 max_samples를 훈련 세트의 크기로 지정한다.

In [9]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)

랜덤 포레스트 알고리즘은 편향을 손해보는 대신 분산을 낮추어 전체적으로 더 휼륭한 모델을 만들어낸다.

다음은 BaggingClassifier를 사용해 앞의 RandomForestClassifier과 거의 유사하게 만든 것이다.

In [10]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(max_features="auto", max_leaf_nodes=16),
    n_estimators=500, max_samples=1.0, bootstrap=True, n_jobs=-1)

### 7.4.1 엑스트라 트리
트리를 더욱 무작위하게 만들기 위해 최적의 임곗값을 찾는 대신 후보 특성을 사용해 무작위로 분할한 다음 그중에서 최상의 분할을 선택한다. 이와 같이 극단적으로 무작위한 트리의 랜덤 포레스트를 익스트림 랜덤 트리 앙상블(또는 엑스트라 트리)이라고 부른다. 일반적인 랜덤 포레스트보다 엑스트라 트리가 훨씬 빠르다.

### 7.4.2 특성 중요도
사이킷런은 훈련이 끝난 뒤 특성마다 자동으로 특성의 상대적 중요도를 걔산하여 중요도의 전체 합이 1이 되도록 결괏값을 정규화한다. 이 값은 feature_importances_ 변수에 저장되어 있다.

In [11]:
from sklearn.datasets import load_iris
iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris["data"], iris["target"])
for name, score in zip(iris["feature_names"], rnd_clf.feature_importances_):
    print(name, score)

sepal length (cm) 0.09854734118804773
sepal width (cm) 0.023884073934686653
petal length (cm) 0.4550692811968381
petal width (cm) 0.4224993036804276


랜덤 포레스트는 특히 특성을 선택해야 할 때 어떤 특성이 중요한지 빠르게 확인할 수 있어 매우 편리하다.